In [1]:
import os
print(os.getcwd())
import sys
sys.path.append('/local/scratch/kchan76/sparse-coarse-operator/')
from libs.pde import *
from libs.rotated_laplacian import RotatedLaplacian
from libs.models import BasicNet, MultiHeadedAttention
import torch

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print('device:', device)

/local/scratch/kchan76


/local/scratch/kchan76/sparse-stencil-learning/src/pyamg2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device: cuda:2


# 3-Level Attention 

In [2]:
# seed = 63
list_eps = [1/100]
low_pi = 2
high_pi = 3
theta_low = np.pi/12*low_pi
theta_high = np.pi/12*high_pi
npts = 5
list_theta = np.linspace(theta_low, theta_high, npts)

In [3]:
train_grid_size = 31
k2 = 10
k3 = 10
same_vecs = False
PDE = RotatedLaplacian(train_grid_size,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

In [4]:
# LEVEL 2
seed=63
set_global_seed(seed)
h = 8
dhid = 256
epochs = 1000
adam_decay_rate = 0 # adam weight decay
lr = 1e-4 # learning rate
lr_decay_rate = 1 # learning rate decay
lr_decay_step = 250 # learning rate decay step size
init = 'xavier uniform'

ncopy = 1
single_model = False # KEEP IT FALSE!! if true, only model_prob will be learned
softmax_topk = True
dropout = 0.3
enforce_stencil_symmetry = '+'

model_prob_l2 = MultiHeadedAttention(h, dhid, ncopy=ncopy, init=init, dropout=dropout)
model_value_l2 = MultiHeadedAttention(h ,dhid, ncopy=ncopy, init=init, dropout=dropout)



model_prob_l2,model_value_l2 = PDE.train(PDE.A2_train, 
                                         PDE.s2_train,
                                         PDE.eig_vec2_train,
                                         model_prob_l2,
                                         model_value_l2,
                                         epochs,
                                         adam_decay_rate,
                                         lr,
                                         lr_decay_rate,
                                         lr_decay_step,
                                         device,
                                         single_model=single_model,
                                         softmax_on = softmax_topk,
                                         verbose = True)




torch.Size([1, 8])
torch.Size([1, 8])
tensor([[-0.0052,  0.0008, -0.0142, -0.0278,  0.0121, -0.0490, -0.0057, -0.0018]],
       device='cuda:2', dtype=torch.float64, grad_fn=<MeanBackward1>)


TypeError: unsupported operand type(s) for +: 'Identity' and 'Tensor'

In [62]:
# test level-2 model
test_seed = 14
# for test_seed in range(50):
set_global_seed(test_seed)
print('test seed:', test_seed)
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
test_grid_size = 255
max_levels = 2
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)
accel = None

num_iter_standard, num_iter_learning, thetaList, xs, xl = PDE.test_model(num_test,
                                                            models,
                                                            test_grid_size,
                                                            max_levels,
                                                            epsilon,
                                                            theta,
                                                            device,
                                                            random_test = True,
                                                            single_model=single_model,
                                                            softmax_on = softmax_topk,
                                                            top_accel = accel,
                                                            verbose = True)

test seed: 14
standard stencil iter:    64.8   standard stencil time:   4.3478402376174925
learned stencil iter:    62.2   learned stencil time:   3.220900321006775


In [189]:
print(num_iter_standard)
print(num_iter_learning)

[245, 74, 133, 81, 58, 110, 118, 246, 186, 118]
[245, 75, 133, 83, 72, 118, 118, 246, 187, 182]


In [83]:
print(thetaList)

[2.1936428642673165, 2.242861055382838, 2.250320857584587, 2.2121678867095524, 2.214042612893607, 2.175985142843968, 2.17642663172932, 2.1715891287701963, 2.23335100780533, 2.1981257850804647]


In [171]:
print(xl)

[12.73647927 15.03831676 15.30868081 ... 14.9365858  14.07809416
 12.38552599]


## Level 3

In [5]:
# LEVEL 3
l3_seed = 62
set_global_seed(l3_seed)
h3 = 8
dhid3 = 256
epochs3 = 1000
adam_decay_rate3 = 0 # adam weight decay
lr3 = 1e-4 # learning rate
lr_decay_rate3 = 1 # learning rate decay
lr_decay_step3 = 500 # learning rate decay step size
init3 = 'xavier uniform' 

ncopy3 = 1
single_model3 = False
softmax_topk3 = True
dropout3 = 0.3



model_prob_l3 = MultiHeadedAttention(h3 ,dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)
model_value_l3 = MultiHeadedAttention(h3, dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)

model_prob_l3,model_value_l3 = PDE.train(PDE.A3_train, 
                                         PDE.s3_train,
                                         PDE.eig_vec3_train,
                                         model_prob_l3,
                                         model_value_l3,
                                         epochs3,
                                         adam_decay_rate3,
                                         lr3,
                                         lr_decay_rate3,
                                         lr_decay_step3,
                                         device,
                                         single_model= single_model3,
                                         softmax_on = softmax_topk3,
                                         verbose = True)

 epoch:  0  loss:  tensor(0.1176, device='cuda:2', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0022, device='cuda:2', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0014, device='cuda:2', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0015, device='cuda:2', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  999  loss:  tensor(0.0013, device='cuda:2', dtype=torch.float64, grad_fn=<AddBackward0>)


In [43]:
# MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"

In [17]:
MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"
if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)
torch.save(model_prob_l2, MODEL_DIR+"level2_prob.pt")
torch.save(model_value_l2, MODEL_DIR+"level2_value.pt")
torch.save(model_prob_l3, MODEL_DIR+"level3_prob.pt")
torch.save(model_value_l3, MODEL_DIR+"level3_value.pt")

In [31]:
MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"
model_prob_l2=torch.load(MODEL_DIR+"level2_prob.pt")
model_value_l2=torch.load(MODEL_DIR+"level2_value.pt")
model_prob_l3=torch.load(MODEL_DIR+"level3_prob.pt")
model_value_l3 = torch.load(MODEL_DIR+"level3_value.pt")

In [6]:
test_seed = 65
# seed_list_127 =[]
# for test_seed in range(100):
set_global_seed(test_seed)
print('test seed:', test_seed)
sizes = [63,127,255,511]
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
models['level3'] = model_prob_l3, model_value_l3
test_grid_size = 127
max_levels = 3
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)
random_test = True

for test_grid_size in sizes:
    print('size:',test_grid_size)
    set_global_seed(test_seed)
    num_iter_standard, num_iter_learning, thetaList, _, _ = PDE.test_model(num_test,
                                                            models,
                                                            test_grid_size,
                                                            max_levels,
                                                            epsilon,
                                                            theta,
                                                            device,
                                                            random_test=random_test,
                                                            single_model=single_model,
                                                            softmax_on = softmax_topk,
                                                            verbose = True)
#     if np.mean(num_iter_learning) < np.mean(num_iter_standard):
#         seed_list_127.append(test_seed)
#         print("seed list:", seed_list_127)

test seed: 65
size: 63
standard stencil iter:    77.6   standard stencil time:   0.1264270544052124
learned stencil iter:    67.0   learned stencil time:   0.23896803855895996
size: 127
standard stencil iter:    86.7   standard stencil time:   0.8738681077957153
learned stencil iter:    78.6   learned stencil time:   0.9884437084197998
size: 255
standard stencil iter:    90.5   standard stencil time:   2.1650845289230345
learned stencil iter:    86.0   learned stencil time:   2.766158628463745
size: 511
standard stencil iter:    92.1   standard stencil time:   11.1967777967453
learned stencil iter:    89.0   learned stencil time:   12.574031662940978


In [70]:
print(num_iter_standard)
print(num_iter_learning)

[104, 95, 92, 112, 109, 90, 103, 110, 103, 110]
[423, 414, 419, 576, 468, 424, 425, 464, 421, 497]


# MLP

In [72]:
# define problem
list_eps = [0.01]
theta_low = np.pi/6
theta_high = np.pi/4
list_theta = np.linspace(theta_low, theta_high, 10)

In [74]:
# seed += 1
print('seed:',seed)
set_global_seed(seed)
epochs = 2000
adam_decay_rate = 0 # adam weight decay
lr = 1e-3 # learning rate
lr_decay_rate = 0.5 # learning rate decay
lr_decay_step = 500 # learning rate decay step size
model_prob_l2 = BasicNet([64])
model_value_l2 = BasicNet([64])
k2 = 10
k3 = 15
same_vecs = False
single_model = False # if true, only model_prob will be learned. 
softmax_on = True # if true, turn on the softmax in the function top_k during sparsfication
problem = RotatedLaplacian(31,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

model_prob_l2,model_value_l2 = problem.train(problem.A2_train, 
                                             problem.s2_train,
                                             problem.eig_vec2_train,
                                             model_prob_l2,
                                             model_value_l2,
                                             epochs,
                                             adam_decay_rate,
                                             lr,
                                             lr_decay_rate,
                                             lr_decay_step,
                                             device,
                                             single_model=single_model,
                                             softmax_on = softmax_on,
                                             verbose = True)

seed: 36
 epoch:  0  loss:  tensor(6.7295, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0111, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0104, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0101, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(0.0098, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1250  loss:  tensor(0.0096, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1500  loss:  tensor(0.0095, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1750  loss:  tensor(0.0094, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1999  loss:  tensor(0.0093, device='cuda:0', grad_fn=<AddBackward0>)
